In [2]:
!pip install chess pandarallel

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 7.8 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=f7b387cb86f4305fa59210f96a9aeb775a5c24a5e6f6e4b0e6bec55db876583d
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [3]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from pandarallel import pandarallel
from multiprocessing import Pool
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import chess
import time 

random.seed(42)
np.random.seed(42)

In [4]:
values = {
    "P": 1,
    "R": 5,
    "B": 3,
    "N": 3,
    "K": 0,
    "Q": 9,
    "p": -1,
    "r": -5,
    "b": -3,
    "n": -3,
    "k": 0,
    "q": -9
}

def tan_eval_categories(evaluation):
    if evaluation < -.5:
        return 0
    if evaluation > .5:
        return 1
    return 2

def eval_to_int(evaluation):
    return int(evaluation) / 100

In [ ]:
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

In [ ]:
df = pd.read_csv("/kaggle/input/no-check-mate-fen/final_data.csv")

In [ ]:
count_pieces = lambda fen, pat=re.compile(r"[a-zA-Z]"): len(re.findall(pat, fen.split(' ')[0]))
transform_eval = lambda x: np.arctan(x / 3)
untransform_eval = lambda x: 3 * np.tan(x)

In [ ]:
df["Pieces"] = df["FEN"].parallel_map(count_pieces)
df["Evaluation"] = df["Evaluation"].parallel_map(eval_to_int)
df["Normalized Evaluation"] = transform_eval(df["Evaluation"])
df["Normalized WLD"] = df["Normalized Evaluation"].parallel_map(tan_eval_categories)

In [6]:
piece_to_layer = {
        'R': 1,
        'N': 2,
        'B': 3,
        'Q': 4,
        'K': 5,
        'P': 6,
        'p': 7,
        'k': 8,
        'q': 9,
        'b': 10,
        'n': 11,
        'r': 12
}

In [25]:
def to_bitboard(fen):
    boards = np.zeros((29, 8, 8), dtype=np.uint8)
    board = chess.Board(fen)
    color = bool(board.turn)
    cr = board.castling_rights
    wkcastle = bool(cr & chess.BB_H1)
    wqcastle = bool(cr & chess.BB_A1)
    bkcastle = bool(cr & chess.BB_H8)
    bqcastle = bool(cr & chess.BB_A8)

    boards[0, :, :]  = color
    boards[25, :, :] = wkcastle
    boards[26, :, :] = wqcastle
    boards[27, :, :] = bkcastle
    boards[28, :, :] = bqcastle

    piece_map = board.piece_map()
    for i, p in piece_map.items():
        rank, file = to_square(i)
        piece = p.symbol()
        boards[piece_to_layer[piece], rank, file] = 1
        for sq in board.attacks(i):
            attack_rank, attack_file = to_square(sq)
            boards[piece_to_layer[piece] + 12, attack_rank, attack_file] = 1
    return boards

def to_square(number):
    rank, file = divmod(number, 8)
    return 7 - rank, file

# In bitboard
def print_bitboard(boards):
    np.set_printoptions(linewidth=150)
    for i in range(boards.shape[0]):
        print(f"Layer {i}:")
        print(boards[i])
        print("\n")

print_bitboard(to_bitboard("r3k2r/p1ppqpb1/bn2pnp1/3PN3/1p2P3/2N2Q1p/PPPBBPPP/R3K2R w KQkq - 0 1"))

Layer 0:
[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]


Layer 1:
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1]]


Layer 2:
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


Layer 3:
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]]


Layer 4:
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


Layer 5:
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]]


Layer 6:
[[0 0 0

In [ ]:
def safe_sample(group, n):
    return group.sample(n=n) if len(group) >= n else group

bdf = df[df["Pieces"] >= 5].groupby("Pieces").apply(lambda x: safe_sample(x, 10000000)).reset_index(drop=True)

# Áp dụng lại cho các nhóm "Normalized WLD"
bbdf = bdf.groupby("Normalized WLD").apply(lambda x: safe_sample(x, 2000000)).reset_index(drop=True)

In [ ]:
l = len(bbdf)
bbdf = bbdf.sample(frac=1)
train_df, test_df, cv_df = bbdf[:int(.8 * l)], bbdf[int(.8 * l): int(.9 * l)], bbdf[int(.9 * l):]

In [ ]:
import gc
gc.collect()

In [ ]:
class ChessDataset(Dataset):
    def __init__(self, df):
        self.fens = torch.from_numpy(np.array([*map(to_bitboard, df["FEN"])], dtype=np.uint8))
        self.evals = torch.Tensor([[x] for x in df["Normalized Evaluation"]])
        self._len = len(self.evals)
        
    def __len__(self):
        return self._len
    
    def __getitem__(self, index):
        return self.fens[index], self.evals[index]

d_train, d_test, d_cv = map(ChessDataset, [train_df, test_df, cv_df])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = nn.Sequential(
    nn.Conv2d(29, 128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.Conv2d(128, 256, kernel_size=3, stride=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.Dropout(0.4),
    nn.Conv2d(256, 1024, kernel_size=2, stride=1),
    nn.ReLU(),
    nn.BatchNorm2d(1024),
    nn.Dropout(0.4),
    nn.Conv2d(1024, 512, kernel_size=2, stride=1),
    nn.ReLU(),
    nn.BatchNorm2d(512),
    nn.Dropout(0.4),
    nn.Conv2d(512, 256, kernel_size=3, stride=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.Dropout(0.4),
    nn.Flatten(),
    nn.Linear(1024, 512),
    nn.Dropout(0.2),
    nn.Linear(512, 1) 
).to(device)

def init_weights(m):
    try:
        nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    except Exception:
        return

model.apply(init_weights)

In [ ]:
# Đọc lại mảng từ file
train_losses_read = np.loadtxt('/kaggle/input/no-check-mate-fen/train_losses.txt')
val_losses_read = np.loadtxt('/kaggle/input/no-check-mate-fen/val_losses.txt')

# Chuyển đổi mảng numpy thành list
train_losses_read = train_losses_read.tolist()
val_losses_read = val_losses_read.tolist()

In [ ]:
#train_losses, val_losses = [], []
train_losses = train_losses_read
val_losses = val_losses_read
print(train_losses)
print(val_losses)
best_val_loss = 0.12899917476096856
epoch_offset = 18
patience = 10
counter = 1
checkpoint_path = 'best_model.pth'

In [ ]:
model.load_state_dict(torch.load('/kaggle/input/no-check-mate-fen/best_model.pth'))

In [ ]:
batch_size = 128
num_work = 2
print("Converting to pytorch Dataset...")
train_loader = DataLoader(dataset=d_train, batch_size=batch_size, shuffle=True, num_workers=num_work)
cv_loader = DataLoader(dataset=d_cv, batch_size=batch_size, shuffle=True, num_workers=num_work)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0003) 
print("Done")

In [ ]:
# import time
# import torch

# train_losses, val_losses = [], []

# best_val_loss = float('inf')
# patience = 10
# counter = 0
# checkpoint_path = 'best_model.pth'

# for epoch in range(1000):
#     start_time = time.time()
#     running_loss = []

#     model.train()
#     for data, target in train_loader:
#         optimizer.zero_grad()
#         data, target = data.to(device), target.to(device)
#         y_pred = model(data.float())
#         loss = criterion(y_pred, target)
        
#         running_loss.append(loss.item())
#         loss.backward()
#         optimizer.step()

#     # Calculate average training loss for the epoch
#     train_loss = sum(running_loss) / len(running_loss)
#     train_losses.append(train_loss)

#     # Print training loss for the epoch
#     print(f"[TRAIN] epoch: {epoch + 1:5}, loss: {train_loss:10}, time: {time.time() - start_time:.2f} seconds", end="\t")

#     # Validation loop
#     model.eval()  # Set the model to eval mode
#     running_loss = []
#     with torch.no_grad():
#         for data, target in cv_loader:
#             data, target = data.to(device), target.to(device)
#             y_pred = model(data.float())
#             loss = criterion(y_pred, target)
#             running_loss.append(loss.item())
        
#     # Calculate average validation loss for the epoch
#     val_loss = sum(running_loss) / len(running_loss)
#     val_losses.append(val_loss)

#     # Print validation loss for the epoch
#     print(f"[VAL] epoch: {epoch + 1:5}, loss: {val_loss:10}, time: {time.time() - start_time:.2f} seconds")

#     # Check early stopping condition and save model if validation loss improves
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         counter = 0
#         torch.save(model.state_dict(), checkpoint_path)
#         print(f"Validation loss decreased ({val_loss:.6f}). Saving model ...")
#     else:
#         counter += 1
#         if counter >= patience:
#             print(f"Early stopping triggered at epoch {epoch + 1}")
#             break

# print('Finished Training')
# # Load the last checkpoint with the best model
# model.load_state_dict(torch.load(checkpoint_path))
# torch.save(model.state_dict(), 'final_model.pth')
# print('Finished Training and Best Model Saved')

In [ ]:
for epoch in range(1000):
    start_time = time.time()
    running_loss = []

    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        data, target = data.to(device), target.to(device)
        y_pred = model(data.float())
        loss = criterion(y_pred, target)
        
        running_loss.append(loss.item())
        loss.backward()
        optimizer.step()

    # Calculate average training loss for the epoch
    train_loss = sum(running_loss) / len(running_loss)
    train_losses.append(train_loss)

    # Print training loss for the epoch
    print(f"[TRAIN] epoch: {epoch + epoch_offset + 1 : 5}, loss: {train_loss:10}, time: {time.time() - start_time:.2f} seconds", end="\t")

    # Validation loop
    model.eval()  # Set the model to eval mode
    running_loss = []
    with torch.no_grad():
        for data, target in cv_loader:
            data, target = data.to(device), target.to(device)
            y_pred = model(data.float())
            loss = criterion(y_pred, target)
            running_loss.append(loss.item())
        
    # Calculate average validation loss for the epoch
    val_loss = sum(running_loss) / len(running_loss)
    val_losses.append(val_loss)

    # Print validation loss for the epoch
    print(f"[VAL] epoch: {epoch + epoch_offset + 1:5}, loss: {val_loss:10}, time: {time.time() - start_time:.2f} seconds")

    # Check early stopping condition and save model if validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Validation loss decreased ({val_loss:.6f}). Saving model ...")
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping triggered at epoch {epoch + epoch_offset + 1}")
            break

print('Finished Training')
# Load the last checkpoint with the best model
model.load_state_dict(torch.load(checkpoint_path))
torch.save(model.state_dict(), 'final_model.pth')
print('Finished Training and Best Model Saved')

In [ ]:
# Plotting the losses
epochs = range(1, len(train_losses) + 1)
plt.plot(epochs, train_losses, 'b', label='Training loss')
plt.plot(epochs, val_losses, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np

# Chuyển đổi list thành mảng numpy
train_losses_np = np.array(train_losses)
val_losses_np = np.array(val_losses)

# Lưu mảng vào file
np.savetxt('train_losses.txt', train_losses_np)
np.savetxt('val_losses.txt', val_losses_np)

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
from torch.utils.data import DataLoader, Subset

num_samples = 8000
indices = list(range(num_samples))
d_test_subset = Subset(d_test, indices)
test_loader = DataLoader(dataset=d_test_subset, batch_size=128, shuffle=False, num_workers=2)

# Load the best model
model.load_state_dict(torch.load(checkpoint_path))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Evaluate on test data
running_loss = []
all_preds = []
all_labels = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        y_pred = model(data.float())
        
        loss = criterion(y_pred, target)
        running_loss.append(loss.item())
        
        all_preds.extend(y_pred.cpu().numpy().flatten())
        all_labels.extend(target.cpu().numpy().flatten())

# Calculate average test loss
test_loss = sum(running_loss) / len(running_loss)
print(f"[TEST] loss: {test_loss:10}")

# Calculate other regression metrics
mse = mean_squared_error(all_labels, all_preds)
mae = mean_absolute_error(all_labels, all_preds)
r2 = r2_score(all_labels, all_preds)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")

# Check the lengths of labels and predictions
print(f"Length of all_labels: {len(all_labels)}")
print(f"Length of all_preds: {len(all_preds)}")

# Plot the results
plt.figure(figsize=(10, 7))
plt.scatter(all_labels, all_preds, alpha=0.3)
plt.plot([min(all_labels), max(all_labels)], [min(all_labels), max(all_labels)], color='red', linestyle='--')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values')
plt.show()

In [ ]:
import numpy as np

np.savez('predictions.npz', labels=all_labels, predictions=all_preds)